<a href="https://colab.research.google.com/github/Andrvz/Pos_Graduacao_MVP/blob/main/MVP_2_Olist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

r'F:\Meu Drive\MVP-II\data\raw\olist_geolocation_dataset.csv'
r'F:\Meu Drive\MVP-II\data\raw\product_category_name_translation.csv'

'F:\\Meu Drive\\MVP-II\\data\\raw\\product_category_name_translation.csv'

# MVP - Análise de Dados de Vendas por E-comerce da Olist
#### Desenvolvido por André Vaz em Julho de 2023
\\


**Disciplina:** Machine Learning & Analytics \\
**Pós-Graduação:** Ciência de Dados e Analytics \\
**Universidade:** PUC-Rio


________________________________________________________________________________
## Apresentação do Projeto

- A Olist é uma empresa brasileira de e-commerce que oferece uma plataforma de vendas para lojas de todos os portes. O dataset divulgado de forma pública pela mesma contém um conjunto de tabelas que reunem informações de seus clientes, vendas, pedidos, produtos e vendedores. Abordando o período entre os anos de 2016 à 2018, o dataset da Olist apresenta mais de 100.000 pedidos realizados por cerca de 32.000 clientes em vendas ditribuídas pelas 5 regiões do Brasil.

- O projeto abordará o processo de extração, tratamento e carregamento do dataset. Assim, permitirá o desenvolvimento de estudos que utilizem técnicas de Machine Learning, como a análise de séries temporais e avaliação do perfil de clientes a fim de identificar padrões de compras e comportamentos dos consumidores, assim como fornecer insights para melhorias em marketing e atendimento ao cliente. Devido à limitação de memória para arquivos inseridos no repositório github, este projeto não abordará o dataset de geolocalização de vendas da Olist.

*   Dataset **Customers**: dados sobre clientes.
> **customer_id**: identificação do cliente por pedido. \
> **customer_unique_id**: identificação única do cliente. \
> **customer_zip_code_prefix**: código postal do cliente. \
> **customer_city**: cidade onde o cliente reside. \
> **customer_state**: estado onde o cliente reside.

*   Dataset **Orders**: dados sobre vendas.
> **order_id**: identificação única do pedido. \
> **customer_id**: identificação única do cliente que fez o pedido. \
> **order_status**: status do pedido. \
> **order_purchase_timestamp**: data e hora da compra. \
> **order_approved_at**: data e hora em que o pagamento foi aprovado. \
> **order_delivered_carrier_date**: data em que o pedido foi entregue à transportadora. \
> **order_delivered_customer_date**: data em que o pedido foi entregue ao cliente. \
> **order_estimated_delivery_date**: data estimada de entrega.

*   Dataset **Order Items**: dados sobre pedidos por vendas.
> **order_id**: identificação única do pedido. \
> **order_item_id**: identificação única do item do pedido. \
> **product_id**: identificação única do produto. \
> **seller_id**: identificação única do vendedor. \
> **shipping_limit_date**: data limite para o envio do produto. \
> **price**: preço unitário do produto. \
> **freight_value**: valor do frete.

*   Dataset **Order Payments**: dados sobre pagamentos por vendas.
> **order_id**: identificação única do pedido. \
> **payment_sequential**: número de sequência do pagamento em um determinado pedido. \
> **payment_type**: tipo de pagamento utilizado. \
> **payment_installments**: número de parcelas em que o pagamento foi dividido. \
> **payment_value**: valor total pago.

*   Dataset **Order Reviews**: dados sobre avaliações por vendas.
> **review_id**: identificação única da avaliação. \
> **order_id**: identificação única do pedido. \
> **review_score**: pontuação da avaliação (0 a 5). \
> **review_comment_title**: título presente na mensagem de avaliação.\
> **review_comment_message**: mensagem de avaliação.\
> **review_creation_date**: data de criação da avaliação. \
> **review_answer_timestamp**: data em que a resposta à avaliação foi enviada.

*   Dataset **Products**: dados sobre produtos.
> **product_id**: identificação única do produto. \
> **product_category_name**: categoria do produto. \
> **product_name_lenght**: comprimento em caracteres do nome do produto. \
> **product_description_lenght**: comprimento da descrição do produto. \
> **product_photos_qty**: quantidade de fotos do produto. \
> **product_weight_g**: peso do produto em gramas. \
> **product_length_cm**: comprimento do produto em centímetros. \
> **product_height_cm**: altura do produto em centímetros. \
> **product_width_cm**: largura do produto em centímetros.

*   Dataset **Sellers**: dados sobre vendedores.
> **seller_id**: identificação única do vendedor. \
> **seller_zip_code_prefix**: código postal do vendedor. \
> **seller_city**: cidade onde o vendedor está localizado. \
> **seller_state**: estado onde o vendedor está localizado.

________________________________________________________________________________

## Códigos Python

In [ ]:
#@title 1.1 - Instalações Iniciais

!pip install pandas numpy                                                     # Instalando bibliotecas básicas para cicência de dados
!pip install holoviews seaborn bokeh matplotlib                               # Instalando bibliotecas de visualização
!pip install ipywidgets                                                       # Instalando biblioteca para uso de widgets no notebook
!jupyter nbextension enable --py widgetsnbextension                           # Instalando extensão para uso de widgets no notebook
!pip install summarytools                                                     # Instalando biblioteca para widget de resumo de dados

In [3]:
#@title 1.2 - Importações

# Importações gerais
import os
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
from datetime import date, datetime, timedelta
from summarytools import dfSummary
from openpyxl import Workbook

# Importações de bibliotecas de visualização
import holoviews as hv
from holoviews import opts
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from bokeh.models.tools import HoverTool
from ipywidgets import widgets
from IPython.display import display
import missingno as ms # para tratamento de missings

# Ignorar avisos
import warnings
warnings.filterwarnings("ignore")

In [31]:
#@title 1.3 - Métodos utilizados para ETL
class ETL:
    def __init__(self, tipo):
        self.tipo = tipo

        self.dataset_raw = {}
        self.resumo_tabelas_raw = None
        self.resumo_colunas_raw = None

        self.dataset = {}
        self.resumo_tabelas = None
        self.resumo_colunas = None

        self.pipeline()

    def carregar_dados(self):

        self.dataset_raw = {'customer': pd.read_csv(
            'https://raw.githubusercontent.com/Andrvz/mvp_sprint_1/main/Dataset/olist_customers_dataset.csv'),
            'orders': pd.read_csv(
                'https://raw.githubusercontent.com/Andrvz/mvp_sprint_1/main/Dataset/olist_orders_dataset.csv'),
            'order_items': pd.read_csv(
                'https://raw.githubusercontent.com/Andrvz/mvp_sprint_1/main/Dataset/olist_order_items_dataset.csv'),
            'order_payments': pd.read_csv(
                'https://raw.githubusercontent.com/Andrvz/mvp_sprint_1/main/Dataset/olist_order_payments_dataset.csv'),
            'order_reviews': pd.read_csv(
                'https://raw.githubusercontent.com/Andrvz/mvp_sprint_1/main/Dataset/olist_order_reviews_dataset.csv'),
            'products': pd.read_csv(
                'https://raw.githubusercontent.com/Andrvz/mvp_sprint_1/main/Dataset/olist_products_dataset.csv'),
            'sellers': pd.read_csv(
                'https://raw.githubusercontent.com/Andrvz/mvp_sprint_1/main/Dataset/olist_sellers_dataset.csv')}

    def tratar_dados(self):
        self.dataset = {}

        for data in self.dataset_raw.keys():
            if data == 'customer':
                df = self.dataset_raw[f'{data}']
                df = df.astype({'customer_id': 'object',
                                'customer_unique_id': 'object',
                                'customer_zip_code_prefix': 'int',
                                'customer_city': 'category',
                                'customer_state': 'category'})
                df.dropna(inplace=True)
                df.drop(columns=[], inplace=True)
                self.dataset[f'{data}'] = df
                del df
            if data == 'geolocation':
                df = self.dataset_raw[f'{data}']
                # df = df.astype({'customer_id':'object',
                #                 'customer_unique_id':'object',
                #                 'customer_zip_code_prefix':'int',
                #                 'customer_city':'category',
                #                 'customer_state':'category'})

                df.dropna(inplace=True)
                df.drop(columns=[], inplace=True)
                self.dataset[f'{data}'] = df
                del df
            if data == 'order_items':
                df = self.dataset_raw[f'{data}']
                df = df.astype({'order_id': 'object',
                                'order_item_id': 'int',
                                'product_id': 'object',
                                'seller_id': 'object',
                                'shipping_limit_date': 'datetime64[ns]',
                                'price': 'float',
                                'freight_value': 'float'})

                df.dropna(inplace=True)
                df.drop(columns=[], inplace=True)
                self.dataset[f'{data}'] = df
                del df
            if data == 'orders':
                df = self.dataset_raw[f'{data}']

                df = df.assign(
                    purchase_date=pd.to_datetime(df['order_purchase_timestamp']).dt.date,
                    purchase_year=pd.to_datetime(df['order_purchase_timestamp']).dt.year,
                    purchase_month=pd.to_datetime(df['order_purchase_timestamp']).dt.month,
                    purchase_day=pd.to_datetime(df['order_purchase_timestamp']).dt.day_name(),
                    purchase_hour=pd.to_datetime(df['order_purchase_timestamp']).dt.hour)

                def time_period(x):
                    if x >= 5 and x < 12:
                        return "Morning"
                    elif x >= 12 and x < 17:
                        return "Afternoon"
                    elif x >= 17 and x < 21:
                        return "Evening"
                    else:
                        return "Night"

                df["purchase_time"] = df["purchase_hour"].apply(time_period)

                df = df.astype({'order_id': 'object',
                                'customer_id': 'object',
                                'order_status': 'category',
                                'order_purchase_timestamp': 'datetime64[ns]',
                                'order_approved_at': 'datetime64[ns]',
                                'order_delivered_carrier_date': 'datetime64[ns]',
                                'order_delivered_customer_date': 'datetime64[ns]',
                                'order_estimated_delivery_date': 'datetime64[ns]'})

                df.dropna(inplace=True)
                df.drop(columns=[], inplace=True)
                self.dataset[f'{data}'] = df
                del df
            if data == 'order_payments':
                df = self.dataset_raw[f'{data}']
                df = df.astype({'order_id': 'object',
                                'payment_sequential': 'int',
                                'payment_type': 'category',
                                'payment_installments': 'int',
                                'payment_value': 'float'})

                df.dropna(inplace=True)
                df.drop(columns=[], inplace=True)
                self.dataset[f'{data}'] = df
                del df
            if data == 'order_reviews':
                df = self.dataset_raw[f'{data}']
                df = df.astype({'review_id':'object',
                                'order_id':'object',
                                 'review_score':'int',
                                'review_creation_date':'datetime64[ns]',
                                'review_answer_timestamp':'datetime64[ns]'})

                df.dropna(inplace=True)
                df.drop(columns=[], inplace=True)
                self.dataset[f'{data}'] = df
                del df
            if data == 'products':
                df = self.dataset_raw[f'{data}']
                df['product_width_cm'] = df['product_width_cm'].fillna(0)
                df['product_height_cm'] = df['product_height_cm'].fillna(0)
                df['product_length_cm'] = df['product_length_cm'].fillna(0)
                df['product_weight_g'] = df['product_weight_g'].fillna(0)
                df['product_name_lenght'] = df['product_name_lenght'].fillna(0)
                df['product_description_lenght'] = df['product_description_lenght'].fillna(0)
                df['product_photos_qty'] = df['product_photos_qty'].fillna(0)
                df['product_category_name'] = df['product_category_name'].fillna('Sem Categoria')

                df = df.astype({'product_id': 'object',
                                'product_category_name': 'category',
                                'product_name_lenght': 'float',
                                'product_description_lenght': 'float',
                                'product_photos_qty': 'float',
                                'product_weight_g': 'float',
                                'product_length_cm': 'float',
                                'product_height_cm': 'float',
                                'product_width_cm': 'float'})

                df.dropna(inplace=True)
                df.drop(columns=[], inplace=True)
                self.dataset[f'{data}'] = df
                del df
            if data == 'sellers':
                df = self.dataset_raw[f'{data}']
                df = df.astype({'seller_id': 'object',
                                'seller_zip_code_prefix': 'int',
                                'seller_city': 'category',
                                'seller_state': 'category'})

                df.dropna(inplace=True)
                df.drop(columns=[], inplace=True)
                self.dataset[f'{data}'] = df
                del df


    @staticmethod
    def tabela_consolidada(dict_dataset):

        #### Consolidado
        df = dict_dataset['order_items']
        df = pd.merge(df, dict_dataset['products'],on='product_id')
        df = pd.merge(df, dict_dataset['sellers'],on='seller_id')
        df = pd.merge(df, dict_dataset['orders'],on='order_id')
        df = pd.merge(df, dict_dataset['customer'],on='customer_id')
        df = pd.merge(df, dict_dataset['order_reviews'],on='order_id')
        df = pd.merge(df, dict_dataset['order_payments'],on='order_id')
        return df


    @staticmethod
    def resumo_dados(dicionario_tabelas):
        tabelas = []
        titles = []
        resumo_colunas = pd.DataFrame({},)
        for nome_tabela in dicionario_tabelas:
            tabelas.append(dicionario_tabelas[nome_tabela])
            titles.append(nome_tabela)
            df = HUB.resumo_dados(dicionario_tabelas[nome_tabela])[0]
            df["Tabela"] = df.apply(lambda row: nome_tabela, axis=1)
            resumo_colunas = pd.concat([resumo_colunas, df])

        resumo_tabelas = pd.DataFrame({},)
        resumo_tabelas['datasets']= titles
        resumo_tabelas['columns'] = [', '.join([col for col, null in data.isnull().sum().items() ]) for data in tabelas]
        resumo_tabelas['Linhas']= [data.shape[0] for data in tabelas]
        resumo_tabelas['Colunas']= [data.shape[1] for data in tabelas]
        resumo_tabelas['Duplicadas']= [len(data[data.duplicated()]) for data in tabelas]
        resumo_tabelas['Linhas Nulas']= [data.isnull().sum().sum() for data in tabelas]
        resumo_tabelas['Colunas Nulas'] = [', '.join([col for col, null in data.isnull().sum().items() if null > 0]) for data in tabelas]
        resumo_tabelas.style.background_gradient(cmap='YlGnBu')
        return resumo_tabelas, resumo_colunas

    def pipeline(self):
        path = None
        if self.tipo == "Colab":
            print("Ambiente Python: Google Colab")
            path = "/content/drive/MyDrive/MVP-II/data/"

        if self.tipo == "DataSpell":
            print("Ambiente Python: Dataspell")
            path = "data/"

        self.carregar_dados()
        self.tratar_dados()
        self.dataset_raw['consolidado'] = ETL.tabela_consolidada(self.dataset_raw)
        self.resumo_tabelas_raw, self.resumo_colunas_raw = ETL.resumo_dados(self.dataset_raw)
        self.dataset['consolidado'] = ETL.tabela_consolidada(self.dataset)
        self.resumo_tabelas, self.resumo_colunas = ETL.resumo_dados(self.dataset)

        if path != None:
            ETL.salvar_dados_csv(self.dataset_raw, path, tipo="raw")
            ETL.salvar_dados_csv(self.dataset, path, tipo="processed")

    @staticmethod
    def salvar_dados_csv(data, path, tipo=None):
        # Create a directory to store the CSV files
        print("Datasets salvos em " + path + tipo)
        if not os.path.exists(path + tipo):
            os.makedirs(path + tipo)

        for nome_aba, df in data.items():
            # Convert the DataFrame to a CSV file
            filename = f'{path + tipo}/{nome_aba}.csv'
            df.to_csv(filename, index=False)


In [62]:
#@title 1.4 - Métodos utilizados para Snippets
class HUB:
    @staticmethod
    def relatorio_tabelas(df, title):
      null_counts = df.isnull().sum()
      null_counts = null_counts[null_counts > 0]
      null_counts = null_counts.sort_values(ascending=False)

      print(f"""
Tabela: {title}
Linhas: {df.shape[0]}
Colunas: {df.shape[1]}
Total de nulos:  {df.isnull().sum().sum()}
----------------------------------------------
Colunas nulas:
{null_counts}
    """)

    @staticmethod
    def meses_para_numeros(df, coluna, tipo):
        match tipo:
            case 1:
                df.loc[df[coluna] == 'Jan', coluna] = 1
                df.loc[df[coluna] == 'Fev', coluna] = 2
                df.loc[df[coluna] == 'Mar', coluna] = 3
                df.loc[df[coluna] == 'Abr', coluna] = 4
                df.loc[df[coluna] == 'Mai', coluna] = 5
                df.loc[df[coluna] == 'Jun', coluna] = 6
                df.loc[df[coluna] == 'Jul', coluna] = 7
                df.loc[df[coluna] == 'Ago', coluna] = 8
                df.loc[df[coluna] == 'Set', coluna] = 9
                df.loc[df[coluna] == 'Out', coluna] = 10
                df.loc[df[coluna] == 'Nov', coluna] = 11
                df.loc[df[coluna] == 'Dez', coluna] = 12
                df[coluna] = df[coluna].astype('int64')
                return df
            case 2:
                df.loc[df[coluna] == 'jan', coluna] = 1
                df.loc[df[coluna] == 'fev', coluna] = 2
                df.loc[df[coluna] == 'mar', coluna] = 3
                df.loc[df[coluna] == 'abr', coluna] = 4
                df.loc[df[coluna] == 'mai', coluna] = 5
                df.loc[df[coluna] == 'jun', coluna] = 6
                df.loc[df[coluna] == 'jul', coluna] = 7
                df.loc[df[coluna] == 'ago', coluna] = 8
                df.loc[df[coluna] == 'set', coluna] = 9
                df.loc[df[coluna] == 'out', coluna] = 10
                df.loc[df[coluna] == 'nov', coluna] = 11
                df.loc[df[coluna] == 'dez', coluna] = 12
                df[coluna] = df[coluna].astype('int64')
                return df
            case 3:
                df.loc[df[coluna] == 'Janeiro', coluna] = 1
                df.loc[df[coluna] == 'Fevereiro', coluna] = 2
                df.loc[df[coluna] == 'Março', coluna] = 3
                df.loc[df[coluna] == 'Abril', coluna] = 4
                df.loc[df[coluna] == 'Maio', coluna] = 5
                df.loc[df[coluna] == 'Junho', coluna] = 6
                df.loc[df[coluna] == 'Julho', coluna] = 7
                df.loc[df[coluna] == 'Agosto', coluna] = 8
                df.loc[df[coluna] == 'Setembro', coluna] = 9
                df.loc[df[coluna] == 'Outubro', coluna] = 10
                df.loc[df[coluna] == 'Novembro', coluna] = 11
                df.loc[df[coluna] == 'Dezembro', coluna] = 12
                df[coluna] = df[coluna].astype('int64')
                return df
            case 4:
                df.loc[df[coluna] == 'janeiro', coluna] = 1
                df.loc[df[coluna] == 'fevereiro', coluna] = 2
                df.loc[df[coluna] == 'março', coluna] = 3
                df.loc[df[coluna] == 'abril', coluna] = 4
                df.loc[df[coluna] == 'maio', coluna] = 5
                df.loc[df[coluna] == 'junho', coluna] = 6
                df.loc[df[coluna] == 'julho', coluna] = 7
                df.loc[df[coluna] == 'agosto', coluna] = 8
                df.loc[df[coluna] == 'setembro', coluna] = 9
                df.loc[df[coluna] == 'outubro', coluna] = 10
                df.loc[df[coluna] == 'novembro', coluna] = 11
                df.loc[df[coluna] == 'dezembro', coluna] = 12
                df[coluna] = df[coluna].astype('int64')
                return df

    @staticmethod
    def classe_para_binario(df, coluna, valores):
        df.loc[df[coluna] == valores[0], coluna] = 0
        df.loc[df[coluna] == valores[1], coluna] = 1
        df[coluna] = df[coluna].astype('int64')
        return df

    @staticmethod
    def num_outlier(df_in, col_name):
        q1 = df_in[col_name].quantile(0.05)
        q3 = df_in[col_name].quantile(0.95)
        iqr = q3 - q1  # Interquartile range
        fence_low = q1 - 1.5 * iqr
        fence_high = q3 + 1.5 * iqr
        outliers_df = df_in.loc[(df_in[col_name] < fence_low) | (df_in[col_name] > fence_high)]
        return print("Number of outliers in {} column: ".format(col_name), len(outliers_df)), print("Indexes: ",
                                                                                                    outliers_df.index)

    @staticmethod
    def remove_outlier(df_in, col_name):
        q1 = df_in[col_name].quantile(0.05)
        q3 = df_in[col_name].quantile(0.95)
        iqr = q3 - q1
        fence_low = q1 - 1.5 * iqr
        fence_high = q3 + 1.5 * iqr
        index_outliers = df_in.loc[(df_in[col_name] < fence_low) | (df_in[col_name] > fence_high)].index
        df_in = pd.DataFrame(df_in.drop(index_outliers.to_list(), axis=0, inplace=True))
        print("Outliers in the {} column have been removed".format(col_name))
        return df_in

    @staticmethod
    def check_skew(df, column):
        from scipy import stats
        skew = stats.skew(df[column])
        skewtest = stats.skewtest(df[column])
        plt.title('Distribution of ' + column)
        sns.distplot(df[column])
        plt.show()
        print("{}'s: Skew: {}, : {}".format(column, skew, skewtest))
        return

    @staticmethod
    def resumo_dados(data):
        """
        :param data: pandas dataframe.
        :return: pandas dataframe com tipo, quantidade, valores distintos, valores nulos (total e porcentagem).
        """
        if isinstance(data, pd.DataFrame):
            data_info = pd.DataFrame({})
            data_info['Nulos'] = round(data.isna().sum(), 2)
            data_info['%_Nulos'] = round((data.isna().sum()) / (data.shape[0]) * 100, 1).astype(str) + '%'
            data_info['Tipo'] = data.dtypes.tolist()
            data_info['Valores_Dist'] = data.nunique().tolist()
            data_info['Itens'] = data.count()

            data_info = data_info.reindex(columns=['Tipo', 'Itens', 'Valores_Dist', 'Nulos', '%_Nulos'])
            data_info = data_info.sort_values(by='%_Nulos', ascending=False)
            return data_info, data_info.reset_index()
        if isinstance(data, dict):
            data_info_dict = pd.DataFrame({})
            for key in data.keys():
                data_info = pd.DataFrame({})
                data_info['Nulos'] = round(key.isna().sum(), 2)
                data_info['%_Nulos'] = round((key.isna().sum()) / (key.shape[0]) * 100, 1).astype(str) + '%'
                data_info['Tipo'] = key.dtypes.tolist()
                data_info['Valores_Dist'] = key.nunique().tolist()
                data_info['Itens'] = key.count()
                data_info['Duplicados'] = data[key].duplicated().sum()

                data_info = data_info.reindex(columns=['Tipo', 'Itens', 'Valores_Dist', 'Nulos', '%_Nulos','Duplicados'])
                data_info_dict = pd.concat([data_info_dict, data_info])
                return data_info_dict, data_info_dict.reset_index()
        if not isinstance(data, (pd.DataFrame, dict)):
            print("Insira um argumento do tipo 'pd.DataFrame' ou 'dict'")

    @staticmethod
    def resumo_dados_ruim(data):
      """
      :param data: pandas dataframe or dictionary of pandas dataframes.
      :return: pandas dataframe(s) with type, quantity, distinct values, null values (total and percentage),
              number of duplicated values, percentage of duplicated values, standard deviation, mean, minimum,
              and maximum values.
      """
      if isinstance(data, pd.DataFrame):
          data_info = pd.DataFrame({})
          data_info['Nulos'] = round(data.isna().sum(), 2)
          data_info['%_Nulos'] = round((data.isna().sum()) / (data.shape[0]) * 100, 1).astype(str) + '%'
          data_info['Tipo'] = data.dtypes.tolist()
          data_info['Valores_Dist'] = data.nunique().tolist()
          data_info['Itens'] = data.count()
          data_info['Duplicados'] = data.duplicated().sum()
          data_info['%_Duplicados'] = round((data.duplicated().sum() / data.count()) * 100, 1).astype(str) + '%'
          data_info['%_Valores_Dist'] = round((data.nunique() / data.count()) * 100, 1).astype(str) + '%'
          data_info['Desvio_Padrao'] = data.std()
          data_info['Media'] = data.mean()
          data_info['Minimo'] = data.min()
          data_info['Maximo'] = data.max()
          data_info = data_info.reindex(columns=['Tipo', 'Itens', 'Valores_Dist', '%_Valores_Dist', 'Nulos', '%_Nulos', 'Duplicados','%_Duplicados','Desvio_Padrao','Media', 'Minimo', 'Maximo'])


          return data_info, data_info.reset_index()

      if isinstance(data, dict):
          data_info_dict = pd.DataFrame({})
          for key in data.keys():
              data_info = pd.DataFrame({})
              data_info['Nulos'] = round(data[key].isna().sum(), 2)
              data_info['%_Nulos'] = round((data[key].isna().sum()) / (data[key].shape[0]) * 100, 1).astype(str) + '%'
              data_info['Tipo'] = data[key].dtypes.tolist()
              data_info['Valores_Dist'] = data[key].nunique().tolist()
              data_info['Itens'] = data[key].count()
              data_info['Duplicados'] = data[key].duplicated().sum()
              data_info['%_Duplicados'] = round((data[key].duplicated().sum() / data[key].count()) * 100, 1).astype(str) + '%'
              data_info['%_Valores_Dist'] = round((data[key].nunique() / data[key].count()) * 100, 1).astype(str) + '%'
              data_info['Desvio_Padrao'] = data[key].std()
              data_info['Media'] = data[key].mean()
              data_info['Minimo'] = data[key].min()
              data_info['Maximo'] = data[key].max()
              data_info = data_info.reindex(columns=['Tipo', 'Itens', 'Valores_Dist', '%_Valores_Dist', 'Nulos', '%_Nulos', 'Duplicados','%_Duplicados','Desvio_Padrao','Media', 'Minimo', 'Maximo'])
              data_info_dict = pd.concat([data_info_dict, data_info])


          return data_info_dict, data_info_dict.reset_index()

      if not isinstance(data, (pd.DataFrame, dict)):
          print("Insira um argumento do tipo 'pd.DataFrame' ou 'dict'")

    @staticmethod
    def resumo_dados_categoricos(data):
        """
        :param data: pd.DataFrame ou pd.Series
        :return: pandas dataframe com frequência e proporção de valores distintos.
        """
        if isinstance(data, pd.DataFrame):
            columns = data.select_dtypes(include=['category']).columns.tolist()
            data_info = pd.DataFrame()
            data_info['Frequência'] = data[columns].apply(lambda x: x.value_counts()).T.stack()
            data_info['Proporção'] = data[columns].apply(
                lambda x: x.value_counts(normalize=True).mul(100).round(1).astype(str) + '%').T.stack()
            return data_info, data_info.reset_index()
        if isinstance(data, pd.Series):
            data_info = pd.DataFrame()
            data_info['Frequência'] = data.value_counts().T
            data_info['Proporção'] = (data.value_counts(normalize=True).mul(100).round(1).astype(str) + '%').T
            return data_info, data_info.reset_index()
        if not isinstance(data, (pd.DataFrame, pd.Series)):
            print("Insira um argumento do tipo 'pd.DataFrame' ou 'pd.Series'")

    @staticmethod
    def boxplot(dataset):
        red_circle = dict(markerfacecolor='red', marker='o', markeredgecolor='white')

        fig, axs = plt.subplots(1, len(dataset.columns), figsize=(20, 8))

        for i, ax in enumerate(axs.flat):
            ax.boxplot(dataset.iloc[:, i], flierprops=red_circle)
            # dataset.iloc[:,i]
            ax.set_title(dataset.columns[i], fontsize=20, fontweight='bold')
            ax.tick_params(axis='y', labelsize=14)

            # Checking if column names are equal to columns we expect to be logarithmic
            if dataset.columns[i] == 'RDEP' or dataset.columns[i] == 'RMED':
                ax.semilogy()

        plt.tight_layout()

    @staticmethod
    def histograma(dataset):

        fig, axs = plt.subplots(1, len(dataset.columns), figsize=(20, 2))

        for i, ax in enumerate(axs.flat):
            ax.hist(dataset.iloc[:, i])
            # dataset.iloc[:,i]
            ax.set_title(dataset.columns[i], fontsize=20, fontweight='bold')
            ax.tick_params(axis='y', labelsize=14)

            # Checking if column names are equal to columns we expect to be logarithmic
            if dataset.columns[i] == 'RDEP' or dataset.columns[i] == 'RMED':
                ax.semilogy()

        plt.tight_layout()

In [65]:
#@title 1.5 - Métodos utilizados para Widgets
class VIZ:
    @staticmethod
    def widget_from_dict(dict_df):
        from ipywidgets import widgets
        from IPython.display import display

        dropdown = widgets.ToggleButtons(
            options=list(dict_df.keys()),
            description='Dataset:',
            disabled=False,
            button_style='success')#,
            # tooltips=['Dataset de clientes da Olist.', 'Dataset de geolozalização dos clientes da Olist.',
                      # 'Dataset das vendas feitas pela Olist.', 'Dataset dos itens vendidos por venda feita pela Olist.',
                      # 'Dataset dos métodos de pagamento das vendas feitas pela Olist.','Dataset de avaliações das vendas feitas pela Olist.',
                      # 'Dataset de produtos vendidos pela Olist.', 'Dataset de vendedores parceiros da Olist.'])

        output_1 = widgets.Output()
        output_2 = widgets.Output()
        output_3 = widgets.Output()
        output_4 = widgets.Output()
        output_5 = widgets.Output()

        tab = widgets.Tab([output_1, output_2,output_3, output_4, output_5])
        tab.set_title(0, 'Colunas')
        tab.set_title(1, 'Amostra de 10 linhas')
        tab.set_title(2, 'Describe')
        tab.set_title(3, 'Resumo da Tabela')
        tab.set_title(4, 'Colunas Categóricas')

        def select(change):
            output_1.clear_output()
            output_2.clear_output()
            output_3.clear_output()
            output_4.clear_output()
            output_5.clear_output()

            if change.new in list(dict_df.keys()):
                with output_1:
                    display(HUB.resumo_dados(dict_df[change.new])[0])
                with output_2:
                    display(dict_df[change.new].sample(10))
                with output_3:
                    display(dict_df[change.new].describe())
                with output_4:
                    display(HUB.relatorio_tabelas(dict_df[change.new],change.new)[0])
                with output_5:
                    display(HUB.resumo_dados_categoricos(dict_df[change.new])[0])

        dropdown.observe(select, names="value")
        return display(dropdown, tab)

    @staticmethod
    def widget_from_df(wid_data):

        output_1 = widgets.Output()
        output_2 = widgets.Output()
        output_3 = widgets.Output()
        output_4 = widgets.Output()

        tab = widgets.Tab([output_1, output_2, output_3, output_4])
        tab.set_title(0, 'Informações')
        tab.set_title(1, 'Primeiras 5 linhas')
        tab.set_title(2, 'Últimas 5 linhas')
        tab.set_title(3, 'Coluna Categóricas')

        with output_1:
            display(HUB.resumo_dados(wid_data)[0])
        with output_2:
            display(wid_data.head(5))
        with output_3:
            display(wid_data.tail(5))
        with output_4:
            display(HUB.resumo_dados_categoricos(wid_data))

        return display(tab)

In [15]:
#@title Resumo de tabelas do dataset

def relatorio_tabelas(df, title):
    null_counts = df.isnull().sum()
    null_counts = null_counts[null_counts > 0]
    null_counts = null_counts.sort_values(ascending=False)

    print(f"""
Tabela: {title}
Linhas: {df.shape[0]}
Colunas: {df.shape[1]}
Total de nulos:  {df.isnull().sum().sum()}
----------------------------------------------
Colunas nulas:
{null_counts}
""")

def relatorio_tabelas_consolidado(df, title):

  print(f"""

Tabela: {title}
Linhas: {Olist_dataset['Consolidado'].shape[0]}
Colunas: {Olist_dataset['Consolidado'].shape[1]}
Total de nulos:  {Olist_dataset['Consolidado'].isnull().sum().sum()}
Duplicados:

----------------------------------------------
Dia primeria venda: {Olist_dataset['Consolidado']['order_purchase_timestamp'].min()}
Dia última venda: {Olist_dataset['Consolidado']['order_purchase_timestamp'].max()}
""")

def create_widgets_and_execute_function(dataset_dict):
    def execute_function(change):
        output.clear_output()
        if change.new:
            selected_key = dropdown_key.value
            with output:
                return print(relatorio_tabelas(df=Olist_dataset[selected_key], title=selected_key))


    output = widgets.Output()

    dropdown_key = widgets.Dropdown(options=list(dataset_dict.keys()), description='Tabela:')

    dropdown_key.observe(execute_function, 'value')

    display(dropdown_key)
    display(output)

In [50]:
#@title Explorando o dataset com Boxplot e Histograma (Ajustar tamanho do print pro usuario)
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display


def hist(dataset):
    dataset_dict = {}
    for n in dataset.keys():
        dataset_dict[n] = dataset[n].columns

    def dados_graficos(df, col, datatype):
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))

        if datatype == 'numeric':
            sns.boxplot(x=col, data=df, ax=ax1, orient='v', color='darkslategrey')
            sns.distplot(df[col], ax=ax2, color='teal').set_title(f'Histograma')
            ax1.set_title('Boxplot', fontsize=14, color='black')
            ax1.set_xlabel('Box', fontsize=14, color='black')
            ax1.set_ylabel(col + ' Values', fontsize=14, color='black')

        elif datatype == 'datetime':
            df[col] = pd.to_datetime(df[col], errors='coerce')
            sns.lineplot(x=col, y=col, data=df, ax=ax1, color='darkslategrey')
            sns.distplot(df[col], ax=ax2, color='teal').set_title(f'Histograma')
            ax1.set_title('Gráfico de Linhas', fontsize=14, color='black')
            ax1.set_xlabel('Data', fontsize=14, color='black')
            ax1.set_ylabel(col + ' Values', fontsize=14, color='black')

        elif datatype == 'category':
            sns.countplot(x=col, data=df, ax=ax1, palette='darkslategrey')
            sns.boxplot(x=col, data=df, ax=ax2, orient='v', color='teal').set_title(f'Boxplot')
            ax1.set_title('Contagem', fontsize=14, color='black')
            ax1.set_xlabel(col, fontsize=14, color='black')
            ax1.set_ylabel('Contagem', fontsize=14, color='black')

        plt.show()

    def update_dropdown(*args):
        selected_key = dropdown_key.value
        selected_type = dropdown_type.value

        # Map the data type strings to corresponding data types
        if selected_type == 'numeric':
            dropdown_value.options = dataset[selected_key].select_dtypes(include=np.number).columns.tolist()
        elif selected_type == 'datetime':
            dropdown_value.options = dataset[selected_key].select_dtypes(include=np.datetime64).columns.tolist()
        elif selected_type == 'category':
            dropdown_value.options = dataset[selected_key].select_dtypes(include='category').columns.tolist()


    def select(change):
        output.clear_output()
        if change.new:
            selected_type = dropdown_type.value
            with output:
               dados_graficos(dataset[dropdown_key.value], col=change.new, datatype=selected_type)





    output = widgets.Output()

    dropdown_key = widgets.Dropdown(options=list(dataset.keys()), description='Tabela:')
    dropdown_type = widgets.Dropdown(options=['numeric', 'datetime', 'category'], description='Tipo:')
    dropdown_value = widgets.Dropdown(description='Coluna:')

    dropdown_key.observe(update_dropdown, 'value')
    dropdown_value.observe(select, names="value")
    dropdown_type.observe(update_dropdown, 'value')

    display(dropdown_key, dropdown_type, dropdown_value)
    display(output)


In [17]:
#@title Widget para printar tabela em dicionario de dataframe
# todo:
def widget(dataset_dict):
    def execute_function(change):
        output.clear_output()
        if change.new:
            selected_key = dropdown_key.value
            with output:
                return print(dataset_dict[selected_key])


    output = widgets.Output()

    dropdown_key = widgets.Dropdown(options=list(dataset_dict.keys()), description='Tabela:')

    dropdown_key.observe(execute_function, 'value')

    display(dropdown_key)
    display(output)

In [18]:
#@title Checklist Sugerido

# 2.1 - Entender e descrever claramente o problema que está sendo resolvido.
# Ok    # todo: 1. Qual é a descrição do problema?
# Ok    # todo: 2. Você tem premissas ou hipóteses sobre o problema? Quais?
# ok    # todo: 3. Que restrições ou condições foram impostas para selecionar os dados?
# ok    # todo: 4. Descreva o seu dataset (atributos, imagens, anotações, etc).


# 2.2 - Realizar operações de preparação dos dados.
# Ok    # todo: 1. Preparação dos dados: dataset em raw e tratado
# !!!   # todo: 2. Separação dos dados: em treino, teste e validação
# !!!   # todo: 3. Avaliar aplicação de validação cruzada
# Ok    # todo: 4. Refinar a quantidade de atributos disponíveis com feature selection


# 2.3 - construir modelos para resolver o problema em questão
# todo: 1. Ajuste inicial de hiperparâmetros
# todo: 2. Análise de underfitting
# todo: 3. Otimização de hiperparâmetros
# todo: 4. Avaliar aplicação de outros métodos
# todo: 5. Pode usar ensembles?


# 2.4 - Analisar o desempenho dos modelos gerados em dados não vistos (com a base de teste)
# todo: 1. Seleção de métricas de avaliação
# todo: 2. Testar modelo com novas bases
# todo: 3. Análise de resultados
# todo: 4. Análise de overfitting
# todo: 5. Conclusão

In [37]:
#@title 2.0 - Inicialização do MVP

# Inicializar o projeto
Olist = ETL(tipo="Collab")

# Print com status após processo de ETL
print(f"""
Olist - Variáveis:
{[attr for attr in Olist.__dict__ if not callable(getattr(Olist, attr))]}

Olist - Tabelas do Dataset_raw:
{Olist.dataset_raw.keys()}

Olist - Tabelas do Dataset:
{Olist.dataset.keys()}
""")


Olist - Variáveis:
['tipo', 'dataset_raw', 'resumo_tabelas_raw', 'resumo_colunas_raw', 'dataset', 'resumo_tabelas', 'resumo_colunas']

Olist - Tabelas do Dataset_raw:
dict_keys(['customer', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers', 'consolidado'])

Olist - Tabelas do Dataset:
dict_keys(['customer', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers', 'consolidado'])



In [ ]:
# Widget com consulta às tabelas criadas
# tabelas_widget = {key: Olist.dataset[key] for key in ['customer', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers']}

VIZ.widget_from_dict({"Consolidado": Olist.dataset['consolidado'],
                      "Consolidado Raw": Olist.dataset_raw['consolidado']})


In [43]:
Olist.resumo_tabelas
Olist.resumo_colunas.query('Tabela == "orders"')

,Tipo,Itens,Valores_Dist,Nulos,%_Nulos,Duplicados,% Duplicados,Tabela
order_id,object,96461,96461,0,0.0%,0,0%,orders
customer_id,object,96461,96461,0,0.0%,0,0%,orders
order_status,category,96461,2,0,0.0%,0,0%,orders
order_purchase_timestamp,datetime64[ns],96461,95939,0,0.0%,0,0%,orders
order_approved_at,datetime64[ns],96461,88272,0,0.0%,0,0%,orders
order_delivered_carrier_date,datetime64[ns],96461,80100,0,0.0%,0,0%,orders
order_delivered_customer_date,datetime64[ns],96461,95649,0,0.0%,0,0%,orders
order_estimated_delivery_date,datetime64[ns],96461,445,0,0.0%,0,0%,orders
purchase_date,object,96461,612,0,0.0%,0,0%,orders
purchase_year,int64,96461,3,0,0.0%,0,0%,orders


In [51]:
tabelas_widget = {key: Olist.dataset[key] for key in ['customer', 'orders', 'order_items', 'order_payments', 'order_reviews', 'products', 'sellers']}

# VIZ.widget_from_dict(tabelas_widget)

hist(Olist.dataset)

Dropdown(description='Tabela:', options=('customer', 'orders', 'order_items', 'order_payments', 'order_reviews…

Dropdown(description='Tipo:', options=('numeric', 'datetime', 'category'), value='numeric')

Dropdown(description='Coluna:', options=(), value=None)

Output()

In [22]:
#@title 2.1 - Explorando o Dataset
# todo:
# Resumo de tabelas do dataset
create_widgets_and_execute_function(tabelas_widget)

# Widget com consulta a tabela consolidado
# VIZ.widget_from_df(Olist_dataset["Consolidado"])

# Sumário das colunas do dataset consolidado
# dfSummary(Olist_dataset['Consolidado'].select_dtypes(include=['int64', 'float64']))

# Visualização de dados nulos
# ms.matrix(Olist_dataset['Consolidado'])

Dropdown(description='Tabela:', options=('Consolidado_raw', 'Consolidado', 'Produtos_Dimensoes'), value='Conso…

Output()

In [23]:
#@title 3.1 - RMF Analysis
# todo: https://www.kaggle.com/code/drindeng/detailed-rfm-analysis-and-k-means-clustering

def RMF_Analysis(df):
  # todo: recência
  present_day = df['order_purchase_timestamp'].max() + timedelta(days=2)
  recency_df= pd.DataFrame(df.groupby(by='customer_unique_id', as_index=False)['order_purchase_timestamp'].max())
  recency_df['Recency']= recency_df['order_purchase_timestamp'].apply(lambda x: (present_day - x).days)
  recency_df = recency_df.sort_values(by='Recency', ascending=False)

  # todo: frequência
  frequency_df = pd.DataFrame(df.groupby(["customer_unique_id"]).agg({"order_id":"nunique"}).reset_index())
  frequency_df.rename(columns={"order_id":"Frequency"}, inplace=True)
  frequency_df = frequency_df.sort_values(by='Frequency', ascending=False)

  # todo: valor monetário
  monetary_df = df.groupby('customer_unique_id', as_index=False)['payment_value'].sum()
  monetary_df.columns = ['customer_unique_id', 'Monetary']
  monetary_df = monetary_df.sort_values(by='Monetary', ascending=False)

  #todo: Resumo de recência, frequência e valor monetário
  RF_df = recency_df.merge(frequency_df, on='customer_unique_id')
  RFM_df = RF_df.merge(monetary_df, on='customer_unique_id').drop(columns='order_purchase_timestamp')
  RFM_estatistica = RFM_df.describe().T


  #todo: Remoção de outliers
  def num_outlier(df_in, col_name):
      q1 = df_in[col_name].quantile(0.05)
      q3 = df_in[col_name].quantile(0.95)
      iqr = q3-q1 #Interquartile range
      fence_low  = q1-1.5*iqr
      fence_high = q3+1.5*iqr
      outliers_df= df_in.loc[(df_in[col_name] < fence_low) | (df_in[col_name] > fence_high)]
      # return print("Number of outliers in {} column: ".format(col_name), len(outliers_df)), print("Indexes: ", outliers_df.index)

  for i in ["Recency", "Frequency", "Monetary"]:
      num_outlier(RFM_df, i)

  #todo: Remoção de outliers nos 5%
  def num_outlier(df_in, col_name):
      q1 = df_in[col_name].quantile(0.05)
      q3 = df_in[col_name].quantile(0.95)
      iqr = q3-q1 #Interquartile range
      fence_low  = q1-1.5*iqr
      fence_high = q3+1.5*iqr
      outliers_df= df_in.loc[(df_in[col_name] < fence_low) | (df_in[col_name] > fence_high)]
      # return print("Number of outliers in {} column: ".format(col_name), len(outliers_df)), print("Indexes: ", outliers_df.index)

  for i in ["Recency", "Frequency", "Monetary"]:
      num_outlier(RFM_df, i)

  RFM_df2= RFM_df.copy()
  RFM_df2 = RFM_df2.set_index('customer_unique_id')

  RFM_df2["recency_score"]  = pd.qcut(RFM_df2['Recency'], 5, labels=[5, 4, 3, 2, 1])
  RFM_df2["frequency_score"]= pd.qcut(RFM_df2['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
  RFM_df2["monetary_score"] = pd.qcut(RFM_df2['Monetary'], 5, labels=[1, 2, 3, 4, 5])
  RFM_df2['RFM_SCORE'] = RFM_df2.recency_score.astype(str)+ RFM_df2.frequency_score.astype(str) + RFM_df2.monetary_score.astype(str)

  seg_map= {
      r'111|112|121|131|141|151': 'Lost customers',
      r'332|322|233|232|223|222|132|123|122|212|211': 'Hibernating customers',
      r'155|154|144|214|215|115|114|113': 'Cannot Lose Them',
      r'255|254|245|244|253|252|243|242|235|234|225|224|153|152|145|143|142|135|134|133|125|124': 'At Risk',
      r'331|321|312|221|213|231|241|251': 'About To Sleep',
      r'535|534|443|434|343|334|325|324': 'Need Attention',
      r'525|524|523|522|521|515|514|513|425|424|413|414|415|315|314|313': 'Promising',
      r'512|511|422|421|412|411|311': 'New Customers',
      r'553|551|552|541|542|533|532|531|452|451|442|441|431|453|433|432|423|353|352|351|342|341|333|323': 'Potential Loyalist',
      r'543|444|435|355|354|345|344|335': 'Loyal',
      r'555|554|544|545|454|455|445': 'Champions'
  }


  RFM_df2['Segment'] = RFM_df2['recency_score'].astype(str) + RFM_df2['frequency_score'].astype(str) + RFM_df2['monetary_score'].astype(str)
  RFM_df2['Segment'] = RFM_df2['Segment'].replace(seg_map, regex=True)

  RFMStats = RFM_df2[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(['mean','median', 'min', 'max', 'count'])
  RFMStats['Ratio']= (100*RFMStats['Monetary']["count"]/RFMStats['Monetary']["count"].sum()).round(2)

  RFM_df_raw = RFM_df
  RFM_df = RFM_df2
  RFM_estatistica = RFMStats


  return RFM_df_raw, RFM_df, RFM_estatistica


Olist_Cluster_RMF = RMF_Analysis(Olist_dataset['Consolidado'][['order_purchase_timestamp', 'customer_unique_id', "order_id", 'payment_value']])

widget({'RFM_df_raw': Olist_Cluster_RMF[0],
        'RFM_df': Olist_Cluster_RMF[1],
        'RFM_estatistica': Olist_Cluster_RMF[2]})


Dropdown(description='Tabela:', options=('RFM_df_raw', 'RFM_df', 'RFM_estatistica'), value='RFM_df_raw')

Output()

In [ ]:
#@title 3.1 - RMF Analysis (histograma) - Criar widget

plt.figure(figsize=(12, 10))
plt.subplot(3, 1, 1); sns.distplot(RFM_df['Recency'])
plt.subplot(3, 1, 2); sns.distplot(RFM_df['Frequency'])
plt.subplot(3, 1, 3); sns.distplot(RFM_df['Monetary'])
plt.show()

In [ ]:
#@title 3.1 - RMF Analysis - Distribuição de segmentos (ordenar)
plt.figure(figsize=(20,8))
#plt.rc('font', size=20)
per= sns.barplot(x=Olist_Cluster_RMF[2]['Ratio'], y=Olist_Cluster_RMF[2].index, data=Olist_Cluster_RMF[2], palette="viridis")
sns.despine(bottom = True, left = True)
for i, v in enumerate(Olist_Cluster_RMF[2]['Ratio']):
    per.text(v, i+.20,"  {:.2f}".format(v)+"%", color='black', ha="left")
per.set_ylabel('Segments', fontsize=25)
per.set(xticks=[])
plt.title('Distribution of Segments', fontsize=35)
plt.show()

In [ ]:
#@title 3.2 - Customer Satisfaction Prediction
# todo: https://towardsdatascience.com/case-study-1-customer-satisfaction-prediction-on-olist-brazillian-dataset-4289bdd20076


#The ‘purchase delivery difference’ column gives us the number of days between the time of purchase and delivery.
#The ‘estimated actual delivery difference’ column gives us the delay or the cut-down in the number of days required for the delivery.
del df
df = Olist_dataset['Consolidado']


# Make sure the columns are in the correct datetime format
df['order_delivered_customer_date'] = pd.to_datetime(Olist_dataset['Consolidado']['order_delivered_customer_date'])
df['order_purchase_timestamp'] = pd.to_datetime(Olist_dataset['Consolidado']['order_purchase_timestamp'])
df['order_estimated_delivery_date'] = pd.to_datetime(Olist_dataset['Consolidado']['order_estimated_delivery_date'])

# Calculate the 'purchase-delivery difference'
intermediate_time = df['order_delivered_customer_date'].dt.date - df['order_purchase_timestamp'].dt.date
df['purchase-delivery difference'] = intermediate_time.dt.days

# Calculate the 'estimated-actual delivery difference'
intermediate_time = df['order_estimated_delivery_date'].dt.date - df['order_delivered_customer_date'].dt.date
df['estimated-actual delivery difference'] = intermediate_time.dt.days


df

In [ ]:
#@title 3.3 - RMF con KNN e 6 clusters (talvez vá remover)
# todo: ML()::RMF::Analysis Segementation
#  todo: In this part we want explain our customers segementation, after we apply best method for segementation with Kmeans with 6 customers segementation based on Sillhoutte Score.

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

Olist_resumo_colunas = Olist.resumo_colunas.query('Tabela == "mvp"')
recency = Olist_dataset['mvp'][['customer_unique_id', 'order_purchase_date']].copy()
recency = recency.groupby('customer_unique_id')['order_purchase_date'].max().reset_index()
recency.columns = ['customer_unique_id', 'last_purchase_timestamp']
# Recency, Frequency, Monetary
recency['purchase_int'] = (recency['last_purchase_timestamp'].max() - recency['last_purchase_timestamp']).dt.days
recency.drop(columns='last_purchase_timestamp', inplace=True)
frequency =pd.DataFrame(Olist_dataset['mvp'].groupby('customer_unique_id')['order_id'].count().reset_index())
monetary =pd.DataFrame(Olist_dataset['mvp'].groupby('customer_unique_id')['payment_value'].sum().reset_index())
# Merge
overall = recency.merge(frequency, on='customer_unique_id')
overall = overall.merge(monetary, on='customer_unique_id')

# Rename columns for better intepretation
overall.rename(columns={ 'purchase_int':'recency', 'num_transaction':'frequency','payment_value':'monetary','order_id':'frequency'}, inplace=True)

scaled_features = overall[['customer_unique_id','recency','frequency','monetary']].copy()

col_names = ['monetary', 'recency','frequency']
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

scaled_features[col_names] = features

# Segmentation Based on RFM with KMeans
kmeans = KMeans(n_clusters=6,random_state= 0)
kmeans.fit(scaled_features[['monetary', 'recency','frequency']])
scaled_features['k_means_clust'] = kmeans.labels_
scaled_features['k_means_clust'].value_counts()

#assign customer segmentation based on overall score
ksegment_lst = scaled_features['k_means_clust']
ksegment_cluster = []

for i in ksegment_lst:
    if i == 1:
        cluster = 'passive'
        ksegment_cluster.append(cluster)
    elif i == 0:
        cluster = 'regular'
        ksegment_cluster.append(cluster)
    elif i == 3:
        cluster = 'occation'
        ksegment_cluster.append(cluster)
    elif i == 4:
        cluster = 'valuable'
        ksegment_cluster.append(cluster)
    elif i == 2:
        cluster = 'loyal'
        ksegment_cluster.append(cluster)
    else:
        cluster = 'best'
        ksegment_cluster.append(cluster)

scaled_features['k_means_segment'] = ksegment_cluster

RMF_1_raw, RMF_1_cluster = overall, scaled_features

________________________________________________________________________________
## Caderno em ajuste

________________________________________________________________________________
## Caderno em ajuste

In [ ]:
#@title Compras por mês

def view_compras_por_mes(df, filtro_datas):
    if len(filtro_datas) != 2 or filtro_datas[0] is None or filtro_datas[1] is None or filtro_datas[0] > filtro_datas[1]:
      print('Selecione um período válido.')
      return

    hv.extension('bokeh')
    filtro = [pd.to_datetime(filtro_datas[0]), pd.to_datetime(filtro_datas[1])]

    df = pd.merge(df['orders'], df['order_items'], on="order_id")
    df = df[(df['order_purchase_timestamp'] >= filtro[0]) & (df['order_purchase_timestamp'] <= filtro[1])]
    vendas = df.groupby(pd.Grouper(key='order_purchase_timestamp', freq='M'))['price'].sum().reset_index()
    vendas['price_milhoes'] = vendas['price'] / 1000000

    calendario = pd.DataFrame({'order_purchase_timestamp': pd.date_range(start=filtro[0], end=filtro[1], freq='M')})
    vendas = calendario.merge(vendas, on='order_purchase_timestamp', how='left')
    vendas['price_milhoes'] = vendas['price_milhoes'].fillna(0)

    area = hv.Area(vendas, 'order_purchase_timestamp', 'price_milhoes').options(fill_alpha=0.3, color='blue')
    curve = hv.Curve(vendas, 'order_purchase_timestamp', 'price_milhoes').options(color='blue')
    vlines = hv.Overlay([hv.VLine(mes).options(line_color='lightgray', line_width=1, line_dash='dashed') for mes in vendas['order_purchase_timestamp']])

    area.opts(xlabel='Data', ylabel='Vendas (em milhões de reais)', title='Compras mensais da Olist')
    curve.opts(xlabel='Data', ylabel='Vendas (em milhões de reais)', title='Compras mensais da Olist')
    return (area * curve * vlines).options(height=400, width=800, ylim=(0, vendas['price_milhoes'].max()))


view_compras_por_mes(dataset, filtro_datas=['1-2016', '12-2018'])

In [ ]:
#@title Avaliações e Compras por mês

def view_compras_avaliacoes(dataset, filtro_datas):
    hv.extension('bokeh')

    df = dataset['order_reviews'].merge(dataset['order_items'], on="order_id").merge(dataset['order_payments'], on="order_id").merge(dataset['orders'], on="order_id")
    df = df[(df['order_purchase_timestamp'] >= filtro_datas[0]) & (df['order_purchase_timestamp'] <= filtro_datas[1])]

    # Agrupar dados por mês e avaliação
    vendas = df.groupby([pd.Grouper(key='order_purchase_timestamp', freq='M'), 'review_score'])['payment_value'].sum().reset_index()

    # Converter valores para milhões de reais
    vendas['price_milhoes'] = vendas['payment_value'] / 1000000

    # Formatando a coluna de datas para exibir apenas mês e ano
    vendas['order_purchase_timestamp'] = vendas['order_purchase_timestamp'].dt.strftime("%m/%y")

    # Criar gráfico de barras empilhadas
    bars = hv.Bars(vendas, ['order_purchase_timestamp', 'review_score'], 'price_milhoes').options(
        width=800, height=400, stacked=True, cmap='Spectral', legend_position='right'
    )
    bars.opts(xlabel='Data', ylabel='Vendas (em milhões de reais)', title='Vendas por avaliação da Olist')

    return bars


view_compras_avaliacoes(dataset, filtro_datas=['1-2016', '12-2018'])